<a href="https://colab.research.google.com/github/helmana/DLeeg/blob/master/2ch%20-%2020s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

    100% |████████████████████████████████| 6.2MB 5.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/e4/be/8b/6b737d272fb91d42e7ac5d61cc93227df7d71944e0e0eb15dd
Successfully built mne


In [2]:

import matplotlib

from matplotlib import pyplot as plt
import numpy as np



import random
import mne
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne import io
from mne import viz
#from mne.datasets import testing
from mne import Epochs, io, pick_types
from mne.event import define_target_events
from mne.time_frequency import psd_welch
print(__doc__)

Automatically created module for IPython interactive environment


In [0]:
#with np.errstate(divide='ignore'):
   # np.float64(1.0) / 0.0

In [18]:
subject_number=20


# load dataset in array
list_raw_fnames = [[0]*2]*subject_number
for x in range(subject_number):
    list_raw_fnames[x] = mne.datasets.eegbci.load_data(x+1,[1,2])

list_rawdata = np.zeros((subject_number,2), dtype='object')

for i in range(subject_number):
    for j in range(2):
        list_rawdata[i][j] = mne.io.read_raw_edf(list_raw_fnames[i][j], preload=True)
print( list_rawdata[:][:])

[........................................] 100.00% (  1.2 MB,  13.4 MB/s) \   
[........................................] 100.00% (  1.2 MB,  15.8 MB/s) \   
[........................................] 100.00% (  1.2 MB,  13.1 MB/s) \   
[........................................] 100.00% (  1.2 MB,  14.3 MB/s) \   
[........................................] 100.00% (  1.2 MB,  13.6 MB/s) \   
[........................................] 100.00% (  1.2 MB,  14.2 MB/s) \   
[........................................] 100.00% (  1.2 MB,  15.7 MB/s) \   
[........................................] 100.00% (  1.2 MB,  10.9 MB/s) \   
[........................................] 100.00% (  1.2 MB,  11.7 MB/s) \   
[........................................] 100.00% (  1.2 MB,  11.8 MB/s) \   
[........................................] 100.00% (  1.2 MB,  16.1 MB/s) |   
[........................................] 100.00% (  1.2 MB,  10.7 MB/s) \   
[........................................] 100.00% (

In [19]:
import math


task_number=1
task_time = 60
sampel_number_per_sec =  160 # sampel rate
total_sampel_number =  sampel_number_per_sec *task_time # 60*160
sample_shift = 4 #step len
window_len= 20

# img for each one
subject_img_number = math.floor((total_sampel_number - sampel_number_per_sec) / sample_shift) +1 - window_len +1
test_start_index=subject_img_number - math.floor(subject_img_number/4) 

train_number =  test_start_index 
test_number = subject_img_number - test_start_index
train_number

1757

In [20]:
ch_number = 2

train_img = np.zeros((train_number*subject_number, ch_number, window_len + 1, 160), dtype = float)
train_label =[]


test_img = np.zeros((test_number*subject_number, ch_number, window_len, 160), dtype = float)
test_label =[]


train_img.shape

(35140, 2, 21, 160)

In [0]:

def normalize_channel_data(ch , i, ch_min, ch_max):
  ch = ((ch - ch_min[i]) / (ch_max[i] - ch_min[i] ) )
  return ch

In [0]:
for s in range(subject_number):
    rawdataChannels_t, times_t=rawdataChannels, times =list_rawdata[s][0][:64,:9600]
    ch_max =[]
    ch_min =[]
    for p in range(len(rawdataChannels_t)):
        ch_max = np.append(ch_max, max(rawdataChannels_t[p])) # max for each cannel
        ch_min = np.append(ch_min, min(rawdataChannels_t[p])) # min for each cannel

    for j in range (subject_img_number):
        for i in range(ch_number):
            for z in range (window_len):
                rawdataChannels, times=rawdataChannels, times =list_rawdata[s][0][i,0+(j+z)*sample_shift:sampel_number_per_sec+(j+z)*sample_shift]

                      # normalize method
                rawdataChannels = normalize_channel_data(rawdataChannels, i, ch_min, ch_max)
                if j <test_start_index :
                    train_img[s*train_number + j][i][z] = rawdataChannels
                    if z==window_len-1 :
                      train_img[s*train_number + j][i][z+1] = s
                      
                      
                else:
                    test_img[s*test_number + j - test_start_index][i][z] = rawdataChannels
        if j >=test_start_index :
          test_label = np.append(test_label, (s))

           

In [0]:
#shaffle train img array
train_img_shuffle = train_img

np.random.shuffle(train_img_shuffle)

In [0]:
train_img_2 = np.zeros((train_number*subject_number, ch_number, window_len, 160), dtype = float)


In [25]:
train_img_2.shape

(35140, 2, 20, 160)

In [0]:
# img label 

train_img_shuffle_len =len(train_img_shuffle)
for i in range(train_img_shuffle_len):
    train_label = np.append(train_label, (train_img_shuffle[i][0][window_len][0] ))
    for j in range (ch_number):
      train_img_2[i][j] = np.delete(train_img_shuffle[i][j], window_len, axis=0)

In [11]:
import keras
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
train_label = to_categorical(train_label, subject_number)
test_label = to_categorical(test_label, subject_number)

In [28]:
# valid & train
x_train =train_img_2[:20000]
y_train =train_label[:20000]

x_valid =train_img_2[20000:]
y_valid =train_label[20000:]

x_train.shape

(20000, 2, 20, 160)

In [29]:
train_label[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [30]:
from keras import layers
from keras import models
from keras import regularizers


model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (ch_number,window_len,160), data_format= "channels_first" ))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))
print(model.output.shape)
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
print(model.output.shape)
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.5))
print(model.output.shape)
model.add(layers.Dense(subject_number, activation = 'softmax'))
print(model.output.shape)

(?, 64, 18, 158)
(?, 32, 9, 158)
(?, 30, 7, 128)
(?, ?)
(?, 512)
(?, 20)


In [0]:

from keras import optimizers

model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr= 1e-4),
              metrics = ['acc'])

In [32]:
history = model.fit(
        x_train,
        y_train,
        epochs = 15,
        batch_size = 10,
        validation_data = (x_valid, y_valid)
)

Train on 20000 samples, validate on 15140 samples
Epoch 1/15
20000/20000 [==============================] - 362s 18ms/step - loss: 1.9985 - acc: 0.3637 - val_loss: 1.1786 - val_acc: 0.6551
Epoch 2/15
20000/20000 [==============================] - 365s 18ms/step - loss: 1.0292 - acc: 0.6594 - val_loss: 0.6913 - val_acc: 0.7956
Epoch 3/15
20000/20000 [==============================] - 375s 19ms/step - loss: 0.7047 - acc: 0.7585 - val_loss: 0.4965 - val_acc: 0.8437
Epoch 4/15
20000/20000 [==============================] - 360s 18ms/step - loss: 0.5411 - acc: 0.8127 - val_loss: 0.4088 - val_acc: 0.8639
Epoch 5/15
20000/20000 [==============================] - 385s 19ms/step - loss: 0.4334 - acc: 0.8474 - val_loss: 0.2621 - val_acc: 0.9152
Epoch 6/15
20000/20000 [==============================] - 379s 19ms/step - loss: 0.3552 - acc: 0.8730 - val_loss: 0.1926 - val_acc: 0.9394
Epoch 7/15
20000/20000 [==============================] - 369s 18ms/step - loss: 0.2991 - acc: 0.8952 - val_loss: 0.

In [33]:
model.evaluate(test_img, test_label)


11700/11700 [==============================] - 51s 4ms/step


[0.3419816870922816, 0.9170085470085471]